In [ ]:
# connect to kaggle
!pip install -q kaggle
from google.colab import files
files.upload() # upload kaggle API Token (kaggle.json)

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"ziyadalrzouq","key":"008320fdd77e95b4a5bb3b4614863404"}'}

In [ ]:
# change the directory and the premission mode
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
# download the dataset
!kaggle competitions download -c arabic-poem-classification


arabic-poem-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# unzip the dataset
!unzip arabic-poem-classification.zip

Archive:  arabic-poem-classification.zip
replace poems.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# import the required libraries

import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

df = pd.read_csv('poems.csv')
df_test = pd.read_csv('test.csv')

In [ ]:
df.head()

,Title,Author,Category,Poem
0,هُنّئتَ بالعيدِ بل هُنّي بكَ العيدُ،,صفي الدين الحلي,العصر الأندلسي,هنءت بالعيد بل هني بك العيد فانت لجود بل ارث ...
1,يا بَديعَ الحُسنِ,المعتمد بن عباد,العصر الأندلسي,يا بديع الحسن والاح سان يا بدر الدياجي يا غزا...
2,وأوانس تدنو إذا اجتديت,الأبيوردي,العصر الأندلسي,وَأَوانِسٍ تَدنو إِذا اِجتَدِيَتْ بِحَديثها وَ...
3,حسبي رضاك من الدهر الذي عتبا,ابن دارج القسطلي,العصر الأندلسي,حسبي رضاك من الدهر الذي عتبا وجود كفيك لحظ ال...
4,ماذا يقول المادحو,ظافر الحداد,العصر الأندلسي,ماذا يقول المادحو نَ وأنت مُخترع الغَرائبْ أع...


In [ ]:
# Combine Title, Author, and Poem into a single column seperated by a space
df['text'] = df['Title'].str.cat(df[['Author', 'Poem']], sep=' ')
df_test['text'] = df_test['Title'].str.cat(df_test[['Author', 'Poem']], sep=' ')
df = df.drop(columns=['Title', 'Author', 'Poem'])
df_test = df_test.drop(columns=['Title', 'Author', 'Poem'])
df

,Category,text
0,العصر الأندلسي,هُنّئتَ بالعيدِ بل هُنّي بكَ العيدُ، صفي الدي...
1,العصر الأندلسي,يا بَديعَ الحُسنِ المعتمد بن عباد يا بديع ال...
2,العصر الأندلسي,وأوانس تدنو إذا اجتديت الأبيوردي وَأَوانِسٍ تَ...
3,العصر الأندلسي,حسبي رضاك من الدهر الذي عتبا ابن دارج القسطلي...
4,العصر الأندلسي,ماذا يقول المادحو ظافر الحداد ماذا يقول المادح...
...,...,...
24995,العصر حديث,في مركب على النيل محمود درويش مركب علي النيل‮....
24996,العصر حديث,المشاكسُ فاطمة ناعوت الي حلمي سالم *** جيفارا...
24997,العصر حديث,دخانها يؤنسني راقصا جبران خليل جبران دخانها ...
24998,العصر حديث,سألت القلب لما فر مني بطرس كرامة سألت القلب لم...


In [ ]:
# encode the labels to integers
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(df['Category'])
df['Category'] = encoder.transform(df['Category'])
df['Category']

0        0
1        0
2        0
3        0
4        0
        ..
24995    4
24996    4
24997    4
24998    4
24999    4
Name: Category, Length: 25000, dtype: int64

In [ ]:
# count each category
df['Category'].value_counts()

0    5000
1    5000
2    5000
3    5000
4    5000
Name: Category, dtype: int64

The following code cells will handle normalization of arabic text using the PyArabic library.

In [ ]:
!pip install pyarabic
import pyarabic.araby as araby
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def preprocess_text(text):
  text = araby.strip_diacritics(text)
  text = araby.normalize_hamza(text)
  text = araby.strip_tatweel(text)
  stop_words = stopwords.words("arabic")
  test = [word for word in text if word not in stop_words]
  return text

In [ ]:
df['text'] = df['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)
df

,Category,text
0,0,هنءت بالعيد بل هني بك العيد، صفي الدين الحلي ...
1,0,يا بديع الحسن المعتمد بن عباد يا بديع الحسن ...
2,0,وءوانس تدنو ءذا اجتديت الءبيوردي وءوانس تدنو ء...
3,0,حسبي رضاك من الدهر الذي عتبا ابن دارج القسطلي...
4,0,ماذا يقول المادحو ظافر الحداد ماذا يقول المادح...
...,...,...
24995,4,في مركب على النيل محمود درويش مركب علي النيل‮....
24996,4,المشاكس فاطمة ناعوت الي حلمي سالم *** جيفارا ...
24997,4,دخانها يءنسني راقصا جبران خليل جبران دخانها ...
24998,4,سءلت القلب لما فر مني بطرس كرامة سءلت القلب لم...


In [ ]:
from keras.preprocessing.text import one_hot
voc_size = 40000
onehot_repr = [one_hot(words, voc_size) for words in df['text']]
test_onehot_repr = [one_hot(words, voc_size) for words in df_test['text']]
onehot_repr[1]

[37496,
 34226,
 17929,
 29385,
 4373,
 1685,
 37496,
 34226,
 17929,
 32800,
 27532,
 37496,
 19677,
 27973,
 37496,
 2362,
 28228,
 39589,
 1443,
 38895,
 10072,
 15563,
 22455,
 6113,
 3805,
 26603,
 18446,
 24114,
 25661]

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sent_length = 50
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
test_embedded_docs = pad_sequences(test_onehot_repr, padding='pre', maxlen=sent_length)
print(embedded_docs)

[[39091 12588 35383 ...   981  6786 26642]
 [    0     0     0 ... 18446 24114 25661]
 [    0     0     0 ... 21542  4593 39089]
 ...
 [    0     0     0 ... 11751 21013 18843]
 [    0     0     0 ... 37888 16019  6682]
 [ 2549 31616 27948 ... 16356 28931 26860]]


In [ ]:
embedded_docs[0]

array([39091, 12588, 35383,  6024, 20449, 29265, 27528,  8364, 14087,
       21443,   807, 21701, 16544, 34226,  4146,  1488, 11751, 14769,
       34226, 14083, 27601,  1550,  5594, 37888, 11967, 22012, 11795,
       39330, 37888, 25056, 11658,  4927, 17298, 10427,   204,  9793,
       37453, 18364, 35426,   177, 11795, 34090,  7388, 32681, 14083,
        9793, 36298,   981,  6786, 26642], dtype=int32)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Bidirectional

embedding_vector_features = 128
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(64, dropout=0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics =['accuracy'])

In [ ]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 50, 128)           5120000   
                                                                 
 lstm_11 (LSTM)              (None, 64)                49408     
                                                                 
 dense_16 (Dense)            (None, 32)                2080      
                                                                 
 dense_17 (Dense)            (None, 5)                 165       
                                                                 
Total params: 5171653 (19.73 MB)
Trainable params: 5171653 (19.73 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
X_final = np.array(embedded_docs)
y_final = keras.utils.to_categorical(df['Category'], 5)
y_final = np.array(y_final)
X_kaggle = np.array(test_embedded_docs) # for kaggle submission
y_final

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
# Create the StratifiedShuffleSplit object
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.10, random_state=42)

# Split the data into training and testing sets
for train_index, test_index in sss.split(X_final, y_final):
    X_train, X_test = X_final[train_index], X_final[test_index]
    y_train, y_test = y_final[train_index], y_final[test_index]

print(X_train.shape, X_test.shape)
print(y_train)


(22500, 50) (2500, 50)
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]]


In [ ]:
model.fit(X_final, y_final, epochs=10, batch_size=64)

Epoch 1/10
391/391 [==============================] - 59s 145ms/step - loss: 1.2063 - accuracy: 0.4730
Epoch 2/10
391/391 [==============================] - 59s 151ms/step - loss: 0.5060 - accuracy: 0.8111
Epoch 3/10
391/391 [==============================] - 57s 147ms/step - loss: 0.1645 - accuracy: 0.9447
Epoch 4/10
391/391 [==============================] - 57s 146ms/step - loss: 0.0550 - accuracy: 0.9834
Epoch 5/10
391/391 [==============================] - 58s 150ms/step - loss: 0.0246 - accuracy: 0.9933
Epoch 6/10
391/391 [==============================] - 58s 148ms/step - loss: 0.0186 - accuracy: 0.9948
Epoch 7/10
391/391 [==============================] - 59s 150ms/step - loss: 0.0150 - accuracy: 0.9961
Epoch 8/10
391/391 [==============================] - 59s 150ms/step - loss: 0.0135 - accuracy: 0.9972
Epoch 9/10
391/391 [==============================] - 59s 151ms/step - loss: 0.0144 - accuracy: 0.9962
Epoch 10/10
391/391 [==============================] - 58s 148ms/step - l

We can see that the data has been equally split according to each category

In [ ]:
kaggle_preds = model.predict(X_kaggle)
predicted_classes = np.argmax(kaggle_preds, axis=1)
print(predicted_classes)

153/153 [==============================] - 2s 12ms/step
[3 4 4 ... 0 4 2]


In [ ]:
labels = encoder.inverse_transform(predicted_classes)
labels

array(['العصر المملوكي', 'العصر حديث', 'العصر حديث', ...,
       'العصر الأندلسي', 'العصر حديث', 'العصر العباسي'], dtype=object)

In [ ]:
df_test['Category'] = labels
df_test = df_test.drop(columns=['text'])
df_test.to_csv("submission.csv", index=False)